# 판례 전처리

## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np
import warnings
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

## 2. 클래스 정의

In [29]:
class PrecedentPrcs:
    
    def __init__(self):
        '''초기화'''
        pass
    
    def handling_null(self, prec_df):
        '''결측치 처리'''
        df = prec_df.copy()
        
        # 1. 판례내용 null인 값 제거
        if df['판례내용'].isna().sum() > 0:
            df = df.dropna(subset=['판례내용'], how='any')
        
        # 2. 사건명이 없는 판례 값 채우기 : '제목없음'
        if df['사건명'].isna().sum() > 0:
            df['사건명'] = df['사건명'].fillna(value='제목없음')
        
        # 3. 사건종류명이 없는 판례 값 채우기 : '기타'
        if df['사건종류명'].isna().sum() > 0:
            df['사건종류명'] = df['사건종류명'].fillna(value='기타')
        
        # 4. 판결유형이 없는 판례 값 채우기 : '미기재'
        if df['판결유형'].isna().sum() > 0:
            df['판결유형'] = df['판결유형'].fillna(value='미기재')
            
        # 5. 참조판례가 없는 판례 값 채우기 : '없음'
        if df['참조판례'].isna().sum() > 0:
            df['참조판례'] = df['참조판례'].fillna(value='없음')
            
        # 6. 선고가 없는 판례 값 채우기 : '미기재'
        if df['선고'].isna().sum() > 0:
            df['선고'] = df['선고'].fillna(value='미기재')
            
        # 7. 판시사항이 없는 판례 값 채우기 : '미기재'
        if df['판시사항'].isna().sum() > 0:
            df['판시사항'] = df['판시사항'].fillna(value='미기재')
            
        # 8. 판결요지가 없는 판례 값 채우기 : '미기재'
        if df['판결요지'].isna().sum() > 0:
            df['판결요지'] = df['판결요지'].fillna(value='미기재')
        
        # 9. 참조조문이 없는 판례 값 채우기 : '미기재'
        if df['참조조문'].isna().sum() > 0:
            df['참조조문'] = df['참조조문'].fillna(value='미기재')
        
        return df
        
    def handling_date(self, prec_df):
        '''선고일자 정리'''
        df = prec_df.copy()
        
        def apply_date(x):
            if len(x) == 8:
                x = x[:4] + '-' + x[4:6] + '-' + x[6:8]
            else:
                x = x[:4] + '-00-00'
            
            return x
        
        # 1. 선고일자 데이터 타입 변경
        df['선고일자'] = df['선고일자'].astype('str')
        
        # 2. 선고일자의 특수문자 제거
        df['선고일자'] = df['선고일자'].apply(lambda x : x[:-2])
        df['선고일자'] = df['선고일자'].str.replace('[^0-9]', '')
        
        # 3. 선고일자의 형식 통일 : ex. 2022-08-18 or 2022
        df['선고일자'] = df['선고일자'].apply(apply_date)

        return df
    
    def handling_duplicate(self, prec_df):
        '''중복 처리'''
        df = prec_df.copy()
        
        # 1. 사건번호 중복 제거
        df = df.drop_duplicates(subset=['사건번호'])
        
        return df
    
    def handling_schars(self, prec_df):
        '''html 처리'''
        df = prec_df.copy()
        
        def apply_reference(x):
            temp = []
            if '<' in x:
                soup = BeautifulSoup(x)
                result = ''
                for i in soup.find_all('a'):
                    temp.append(i.text)
                result = ', '.join(temp)
                return result
            else:
                return x
        
        # 1. html 형식 제거
        df['참조판례'] = df['참조판례'].apply(apply_reference)
        
        return df
    
    def handling_snum(self, prec_df):
        '''법원종류코드 및 사건종류코드 처리'''
        df = prec_df.copy()
        
        df = df.astype({'선고일자' : 'str', '법원종류코드' : 'str', '사건종류코드' : 'str'})
        
        df['법원종류코드'] = df['법원종류코드'].apply(lambda x : x[:-2])
        df['사건종류코드'] = df['사건종류코드'].apply(lambda x : x[:-2])
        
        return df
            
    def sort(self, prec_df, standard, ascending = False):
        '''정렬'''
        df = prec_df.copy()
        
        df.sort_values(standard, ascending=ascending)
        
        return df
    
    def run(self, prec_df):
        df = prec_df.copy()
        
        # 1. 판례 결측치 처리
        df = self.handling_null(df)
        
        # 2. 판례 법원종류코드 및 사건종류 코드 처리
        df = self.handling_snum(df)
        
        # 3. 판례 선고일자 처리
        df = self.handling_date(df)
        
        # 4. 판례 중복 처리
        df = self.handling_duplicate(df)
        
        # 5. html 처리
        df = self.handling_schars(df)
        
        # 6. 판례 정렬 : 디폴트 선고일자 내림차순
        df = self.sort(df, standard='선고일자', ascending=False)
        
        # 7. 인덱스 재설정
        df = df.reset_index(drop=True)
        
        return df

## 3. 테스트

In [3]:
prec_df = pd.read_csv('../data/prec_data.csv', encoding='utf-8-sig')
prec_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,20220415.0,선고,수원고법,400202.0,형사,400102.0,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,NaN,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,20220411.0,자,서울고등법원,400202.0,가사,400103.0,결정,NaN,NaN,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",NaN,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,20220323.0,선고,대구고법,400202.0,민사,400101.0,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",NaN,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,20220322.0,자,대법원,400201.0,가사,400103.0,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),NaN,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",NaN,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,20220317.0,선고,대법원,400201.0,민사,400101.0,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


In [4]:
prec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83520 entries, 0 to 83519
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   판례일련번호  83520 non-null  int64  
 1   사건명     83502 non-null  object 
 2   사건번호    83520 non-null  object 
 3   선고일자    83520 non-null  float64
 4   선고      80588 non-null  object 
 5   법원명     83520 non-null  object 
 6   법원종류코드  83519 non-null  float64
 7   사건종류명   83517 non-null  object 
 8   사건종류코드  83517 non-null  float64
 9   판결유형    82733 non-null  object 
 10  판시사항    71498 non-null  object 
 11  판결요지    63030 non-null  object 
 12  참조조문    70245 non-null  object 
 13  참조판례    39174 non-null  object 
 14  판례내용    83518 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 9.6+ MB


In [30]:
prep_obj = PrecedentPrcs()

In [31]:
prep_df = prep_obj.run(prec_df)

C:\Users\Park Ji Seong\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: The default value of regex will change from True to False in a future version.


In [32]:
prep_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,2022-04-15,선고,수원고법,400202,형사,400102,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,없음,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,2022-04-11,자,서울고등법원,400202,가사,400103,결정,미기재,미기재,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",없음,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,2022-03-23,선고,대구고법,400202,민사,400101,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",없음,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,2022-03-22,자,대법원,400201,가사,400103,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),미기재,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",없음,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,2022-03-17,선고,대법원,400201,민사,400101,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n",,"【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


In [33]:
prep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83318 entries, 0 to 83317
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  83318 non-null  int64 
 1   사건명     83318 non-null  object
 2   사건번호    83318 non-null  object
 3   선고일자    83318 non-null  object
 4   선고      83318 non-null  object
 5   법원명     83318 non-null  object
 6   법원종류코드  83318 non-null  object
 7   사건종류명   83318 non-null  object
 8   사건종류코드  83318 non-null  object
 9   판결유형    83318 non-null  object
 10  판시사항    83318 non-null  object
 11  판결요지    83318 non-null  object
 12  참조조문    83318 non-null  object
 13  참조판례    83318 non-null  object
 14  판례내용    83318 non-null  object
dtypes: int64(1), object(14)
memory usage: 9.5+ MB
